In [2]:
%pip install ipywidgets

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from ipywidgets import interact, widgets

# ==============================================================================
# 1. DATA LOADING & PREPROCESSING
# ==============================================================================
# Load data
try:
    df = pd.read_csv('../data_cleaned/combined_urbanization_life_quality_2008_2020.csv')
except FileNotFoundError:
    print("Error: 'combined_urbanization_life_quality_2008_2020.csv' not found.")
    # Create dummy data for testing if file missing (Remove this in production)
    df = pd.DataFrame() 

# Group by Country to get profiles (Average 2008-2020)
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cols_to_drop = ['Year', 'Country_Code']
clustering_features = [c for c in numeric_cols if c not in cols_to_drop]

country_profiles = df.groupby('Country')[clustering_features].mean().reset_index()

# ==============================================================================
# 2. CLUSTERING (Replicating cluster_countries.py Logic)
# ==============================================================================
# Select features for clustering
features_for_clustering = [
    'urban_pop_perc', 
    'Gini coefficient (2021 prices)', 
    'overall score',  # Peace Index
    'homicide rate'
]

# Drop rows with missing values in these features
cluster_data = country_profiles.dropna(subset=features_for_clustering).copy()

# Normalize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(cluster_data[features_for_clustering])

# Run K-Means (k=2)
kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
cluster_data['Cluster'] = kmeans.fit_predict(X_scaled)

# Determine Labels (Stable vs Volatile)
# We assume the cluster with LOWER Peace Score (Better) is Stable
c0_score = cluster_data[cluster_data['Cluster'] == 0]['overall score'].mean()
c1_score = cluster_data[cluster_data['Cluster'] == 1]['overall score'].mean()

if c0_score < c1_score:
    mapping = {0: 'Stable Urbanizers', 1: 'Volatile Urbanizers'}
    color_map = {'Stable Urbanizers': '#1f77b4', 'Volatile Urbanizers': '#d62728'} # Blue / Red
else:
    mapping = {1: 'Stable Urbanizers', 0: 'Volatile Urbanizers'}
    color_map = {'Stable Urbanizers': '#1f77b4', 'Volatile Urbanizers': '#d62728'}

cluster_data['Cluster_Label'] = cluster_data['Cluster'].map(mapping)

# Merge Clusters back to main profile data
df_final = pd.merge(country_profiles, cluster_data[['Country', 'Cluster_Label']], on='Country', how='inner')

# Create Urban Groups for Visualization 1
df_final['Urban_Group'] = pd.cut(df_final['urban_pop_perc'], 
                                 bins=[-1, 50, 75, 101], 
                                 labels=['Low (<50%)', 'Medium (50-75%)', 'High (>75%)'])

# ==============================================================================
# 3. INTERACTIVE VISUALIZATION SETUP
# ==============================================================================

# List of Indicators to Toggle (Matches your EDA logic)
indicator_options = {
    'Global Peace Index (Overall Score)': 'overall score',
    'Militarization': 'militarisation',
    'Political Instability': 'Political instability',
    'Internal Peace': 'internal peace',
    'Homicide Rate': 'homicide rate',
    'Weapons Exports': 'weapons exports',
    'Police Rate': 'police rate'
}

dropdown = widgets.Dropdown(
    options=indicator_options,
    value='overall score',
    description='Select Indicator:',
    style={'description_width': 'initial'}
)

def update_charts(selected_indicator):
    # --- CHART 1: Bar Chart (Urban Groups Trend) ---
    # Calculate averages by Urban Group
    avg_by_group = df_final.groupby('Urban_Group')[selected_indicator].mean().reset_index()
    
    fig1 = px.bar(
        avg_by_group, 
        x='Urban_Group', 
        y=selected_indicator,
        color='Urban_Group',
        title=f"<b>1. Overall Trend: {selected_indicator} by Urbanization Level</b><br><i>(Does it get better or worse as cities grow?)</i>",
        color_discrete_sequence=px.colors.sequential.Viridis,
        labels={'Urban_Group': 'Urbanization Level', selected_indicator: 'Average Score'}
    )
    fig1.update_layout(height=400, showlegend=False)

    # --- CHART 2: Scatter Plot (The Paradox Split) ---
    # Scatter with Trendlines for Stable vs Volatile
    fig2 = px.scatter(
        df_final, 
        x='urban_pop_perc', 
        y=selected_indicator, 
        color='Cluster_Label',
        trendline='ols', # Add regression lines
        hover_name='Country',
        title=f"<b>2. Deep Dive: {selected_indicator} (Stable vs. Volatile)</b><br><i>(The 'Peace Paradox' - Seeing the Divergence)</i>",
        color_discrete_map=color_map,
        labels={'urban_pop_perc': 'Urban Population (%)', selected_indicator: 'Score (Lower is usually better)'}
    )
    
    # Add Global Trendline (Black dashed)
    # We do this manually to overlay it
    fig2.add_traces(
        px.scatter(df_final, x='urban_pop_perc', y=selected_indicator, trendline='ols')
        .update_traces(line=dict(color='black', dash='dash'), showlegend=True, name='Global Trend (All Countries)')
        .data[1] # Take only the trendline, not points
    )
    
    fig2.update_layout(height=500, legend_title_text='Country Group')

    # Display
    fig1.show()
    fig2.show()

# Run the Interactive Widget
interact(update_charts, selected_indicator=dropdown);


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


interactive(children=(Dropdown(description='Select Indicator:', options={'Global Peace Index (Overall Score)':…

In [3]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from ipywidgets import interact, widgets
import statsmodels.api as sm # Essential for trendlines

# ==============================================================================
# 1. DATA LOADING
# ==============================================================================
# Make sure the CSV file is in the same folder
try:
    df = pd.read_csv('../data_cleaned/combined_urbanization_life_quality_2008_2020.csv')
    print("✅ Data Loaded Successfully.")
except FileNotFoundError:
    print("❌ Error: 'combined_urbanization_life_quality_2008_2020.csv' not found.")
    raise

# Group by Country to get profiles (Average 2008-2020)
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cols_to_drop = ['Year', 'Country_Code']
clustering_features = [c for c in numeric_cols if c not in cols_to_drop]

country_profiles = df.groupby('Country')[clustering_features].mean().reset_index()

# ==============================================================================
# 2. PERFORM CLUSTERING (Re-creating your groups)
# ==============================================================================
# Select features for clustering
features_for_clustering = [
    'urban_pop_perc', 
    'Gini coefficient (2021 prices)', 
    'overall score',  # Peace Index
    'homicide rate'
]

# Drop rows with missing values in these features
cluster_data = country_profiles.dropna(subset=features_for_clustering).copy()

# Normalize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(cluster_data[features_for_clustering])

# Run K-Means (k=2)
kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
cluster_data['Cluster'] = kmeans.fit_predict(X_scaled)

# Determine Labels (Stable vs Volatile)
# We assume the cluster with LOWER Peace Score (Better) is Stable
c0_score = cluster_data[cluster_data['Cluster'] == 0]['overall score'].mean()
c1_score = cluster_data[cluster_data['Cluster'] == 1]['overall score'].mean()

# Assign Labels & Colors
if c0_score < c1_score:
    mapping = {0: 'Stable Urbanizers', 1: 'Volatile Urbanizers'}
else:
    mapping = {1: 'Stable Urbanizers', 0: 'Volatile Urbanizers'}

cluster_data['Cluster_Label'] = cluster_data['Cluster'].map(mapping)

# WCAG Compliant Colors (Blue/Red)
color_map = {'Stable Urbanizers': '#1f77b4', 'Volatile Urbanizers': '#d62728'}

# Merge Clusters back to main profile data
df_final = pd.merge(country_profiles, cluster_data[['Country', 'Cluster_Label']], on='Country', how='inner')

# Create Urban Groups for Visualization 1
df_final['Urban_Group'] = pd.cut(df_final['urban_pop_perc'], 
                                 bins=[-1, 50, 75, 101], 
                                 labels=['Low (<50%)', 'Medium (50-75%)', 'High (>75%)'])

# ==============================================================================
# 3. INTERACTIVE DASHBOARD
# ==============================================================================

# List of Indicators to Toggle
indicator_options = {
    'Global Peace Index (Overall Score)': 'overall score',
    'Militarization': 'militarisation',
    'Political Instability': 'Political instability',
    'Internal Peace': 'internal peace',
    'Homicide Rate': 'homicide rate',
    'Weapons Exports': 'weapons exports',
    'Police Rate': 'police rate',
    'Inequality (Gini)': 'Gini coefficient (2021 prices)'
}

dropdown = widgets.Dropdown(
    options=indicator_options,
    value='overall score',
    description='Select Indicator:',
    style={'description_width': 'initial'}
)

def update_charts(selected_indicator):
    # --- CHART 1: Bar Chart (Urban Groups Trend) ---
    # Fix: Added observed=True to handle FutureWarnings
    avg_by_group = df_final.groupby('Urban_Group', observed=True)[selected_indicator].mean().reset_index()
    
    fig1 = px.bar(
        avg_by_group, 
        x='Urban_Group', 
        y=selected_indicator,
        color='Urban_Group',
        title=f"<b>1. Overall Trend: {selected_indicator} by Urbanization Level</b><br><i>(Does it get better or worse as cities grow?)</i>",
        color_discrete_sequence=px.colors.sequential.Viridis,
        labels={'Urban_Group': 'Urbanization Level', selected_indicator: 'Average Score'}
    )
    fig1.update_layout(height=400, showlegend=False)

    # --- CHART 2: Scatter Plot (The Paradox Split) ---
    # Scatter with Trendlines for Stable vs Volatile
    fig2 = px.scatter(
        df_final, 
        x='urban_pop_perc', 
        y=selected_indicator, 
        color='Cluster_Label',
        trendline='ols', # REQUIRES statsmodels
        hover_name='Country',
        title=f"<b>2. Deep Dive: {selected_indicator} (Stable vs. Volatile)</b><br><i>(The 'Peace Paradox' - Seeing the Divergence)</i>",
        color_discrete_map=color_map,
        labels={'urban_pop_perc': 'Urban Population (%)', selected_indicator: 'Score (Lower is usually better)'}
    )
    
    # Add Global Trendline (Black dashed) manually
    global_trend = px.scatter(df_final, x='urban_pop_perc', y=selected_indicator, trendline='ols')
    if len(global_trend.data) > 1:
        trace = global_trend.data[1]
        trace.line.color = 'black'
        trace.line.dash = 'dash'
        trace.name = 'Global Trend (All Countries)'
        trace.showlegend = True
        fig2.add_trace(trace)
    
    fig2.update_layout(height=500, legend_title_text='Country Group')

    # Display
    fig1.show()
    fig2.show()

# Run the Interactive Widget
print("✨ Generating Dashboard...")
interact(update_charts, selected_indicator=dropdown);

✅ Data Loaded Successfully.
✨ Generating Dashboard...


interactive(children=(Dropdown(description='Select Indicator:', options={'Global Peace Index (Overall Score)':…

In [5]:
# ==============================================================================
# 0. SETUP & INSTALLATION (Ensures Standalone Capability)
# ==============================================================================
import sys
import subprocess
import importlib

# List of required libraries
required_libs = ['pandas', 'numpy', 'plotly', 'scikit-learn', 'ipywidgets', 'statsmodels']

for lib in required_libs:
    if importlib.util.find_spec(lib) is None:
        print(f"Installing missing library: {lib}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", lib])

print("✅ Libraries verified.")

# ==============================================================================
# 1. IMPORTS
# ==============================================================================
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from ipywidgets import interact, widgets
import statsmodels.api as sm

# ==============================================================================
# 2. DATA LOADING & PREPROCESSING
# ==============================================================================
try:
    df = pd.read_csv('../data_cleaned/combined_urbanization_life_quality_2008_2020.csv')
    print(f"✅ Data Loaded: {len(df)} rows.")
except FileNotFoundError:
    print("❌ Error: 'combined_urbanization_life_quality_2008_2020.csv' not found. Please ensure the file is in the same directory.")
    raise

# Create Country Profiles (Average 2008-2020)
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cols_to_drop = ['Year', 'Country_Code']
profile_features = [c for c in numeric_cols if c not in cols_to_drop]
country_profiles = df.groupby('Country')[profile_features].mean().reset_index()

# ==============================================================================
# 3. CLUSTERING (Replicating the Logic)
# ==============================================================================
# Features used for the "Stable vs Volatile" split
cluster_features = ['urban_pop_perc', 'Gini coefficient (2021 prices)', 'overall score', 'homicide rate']
cluster_data = country_profiles.dropna(subset=cluster_features).copy()

# Normalize & Cluster
scaler = StandardScaler()
X_scaled = scaler.fit_transform(cluster_data[cluster_features])
kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
cluster_data['Cluster'] = kmeans.fit_predict(X_scaled)

# Assign Labels (Lower Peace Score = Stable)
c0_score = cluster_data[cluster_data['Cluster'] == 0]['overall score'].mean()
c1_score = cluster_data[cluster_data['Cluster'] == 1]['overall score'].mean()

if c0_score < c1_score:
    mapping = {0: 'Stable Urbanizers', 1: 'Volatile Urbanizers'}
else:
    mapping = {1: 'Stable Urbanizers', 0: 'Volatile Urbanizers'}

cluster_data['Cluster_Label'] = cluster_data['Cluster'].map(mapping)

# WCAG Compliant Colors
color_map = {'Stable Urbanizers': '#1f77b4', 'Volatile Urbanizers': '#d62728'} # Blue / Red

# Merge back to main dataframe
df_final = pd.merge(country_profiles, cluster_data[['Country', 'Cluster_Label']], on='Country', how='inner')

# Create Urban Groups (Low/Med/High) for Vis 1
df_final['Urban_Group'] = pd.cut(df_final['urban_pop_perc'], 
                                 bins=[-1, 50, 75, 101], 
                                 labels=['Low (<50%)', 'Medium (50-75%)', 'High (>75%)'])

# ==============================================================================
# 4. INDICATOR SELECTION (Top 5 Positive & Top 5 Negative)
# ==============================================================================
# Calculate correlations with Urbanization
correlations = df_final[profile_features].corrwith(df_final['urban_pop_perc']).sort_values()

# Remove 'urban_pop_perc' itself from the list
correlations = correlations.drop('urban_pop_perc', errors='ignore')

# 1. Select Top 5 Negative (Decreasing with Urbanization -> "Improving" usually)
top_negative = correlations.head(5).index.tolist()

# 2. Select Top 5 Positive (Increasing with Urbanization -> "Worsening" or "Growing")
top_positive = correlations.tail(5).index.tolist()

# Ensure 'overall score' (Global Peace Index) is included
target_metric = 'overall score'
if target_metric not in top_negative and target_metric not in top_positive:
    # If not in top 5, force add it (replacing the weakest correlation in its group)
    corr_val = correlations[target_metric]
    if corr_val > 0:
        top_positive[-1] = target_metric # Replace last positive
    else:
        top_negative[-1] = target_metric # Replace last negative

# Combine List (Negative first, then Positive)
selected_indicators = top_negative + top_positive
print(f"✅ Selected 10 Indicators:\n  Negative Correl (Decreasing): {top_negative}\n  Positive Correl (Increasing): {top_positive}")

# ==============================================================================
# 5. VISUALIZATION 1 PREP (Normalization)
# ==============================================================================
# We need to normalize these 10 indicators to 0-1 scale so they fit on one bar chart
vis1_data = df_final[['Urban_Group'] + selected_indicators].copy()

# MinMax Scaling
min_max_scaler = MinMaxScaler()
vis1_data[selected_indicators] = min_max_scaler.fit_transform(vis1_data[selected_indicators])

# Melt for Grouped Bar Chart
melted_vis1 = vis1_data.melt(id_vars='Urban_Group', var_name='Indicator', value_name='Normalized Score')
# Calculate Mean per Group
grouped_vis1 = melted_vis1.groupby(['Urban_Group', 'Indicator'], observed=True)['Normalized Score'].mean().reset_index()

# Sort to keep Negative on left, Positive on right
grouped_vis1['Indicator'] = pd.Categorical(grouped_vis1['Indicator'], categories=selected_indicators, ordered=True)
grouped_vis1 = grouped_vis1.sort_values('Indicator')

# ==============================================================================
# 6. DASHBOARD GENERATION
# ==============================================================================

# Create Dropdown Widget
dropdown = widgets.Dropdown(
    options=selected_indicators,
    value='overall score', # Default
    description='Select Detail:',
    style={'description_width': 'initial'}
)

def render_dashboard(selected_indicator):
    # --- VISUALIZATION 1: GROUPED BAR CHART (Overview) ---
    fig1 = px.bar(
        grouped_vis1,
        x='Indicator',
        y='Normalized Score',
        color='Urban_Group',
        barmode='group',
        title='<b>1. Landscape Overview: Top 5 Decreasing vs. Top 5 Increasing Indicators</b><br><i>(Normalized Scores 0-1 across Urbanization Levels)</i>',
        color_discrete_sequence=px.colors.sequential.Viridis,
        height=450
    )
    fig1.update_layout(xaxis_tickangle=-45, legend_title="Urban Level")
    
    # Add annotation to separate the groups
    fig1.add_vline(x=4.5, line_width=2, line_dash="dash", line_color="black")
    fig1.add_annotation(x=2, y=1.05, text="📉 Decreasing (Improving)", showarrow=False, font=dict(color="blue"))
    fig1.add_annotation(x=7, y=1.05, text="📈 Increasing (Worsening/Growing)", showarrow=False, font=dict(color="red"))

    # --- VISUALIZATION 2: SCATTER PLOT (Deep Dive) ---
    # Shows the "Peace Paradox" split for the SELECTED indicator
    fig2 = px.scatter(
        df_final, 
        x='urban_pop_perc', 
        y=selected_indicator, 
        color='Cluster_Label',
        trendline='ols',
        hover_name='Country',
        title=f"<b>2. Deep Dive: {selected_indicator} (Stable vs. Volatile)</b><br><i>(Detailed Correlation Analysis)</i>",
        color_discrete_map=color_map,
        labels={'urban_pop_perc': 'Urban Population (%)', selected_indicator: 'Actual Score'},
        height=500
    )
    
    # Add Global Trendline
    global_trend = px.scatter(df_final, x='urban_pop_perc', y=selected_indicator, trendline='ols')
    if len(global_trend.data) > 1:
        trace = global_trend.data[1]
        trace.line.color = 'black'
        trace.line.dash = 'dash'
        trace.name = 'Global Trend'
        trace.showlegend = True
        fig2.add_trace(trace)

    fig2.update_layout(legend_title="Country Cluster")

    # Display
    fig1.show()
    fig2.show()

# Render
interact(render_dashboard, selected_indicator=dropdown);

Installing missing library: scikit-learn...
✅ Libraries verified.
✅ Data Loaded: 767 rows.
✅ Selected 10 Indicators:
  Negative Correl (Decreasing): ['rural_pop_perc', 'Agriculture, forestry, and fishing, value added (% of GDP)', 'Political instability', 'Adjusted savings: carbon dioxide damage (% of GNI)', 'overall score']
  Positive Correl (Increasing): ['militarisation', 'weapons imports', 'weapons exports', 'clean_fuel_tech_cook_pop', 'Access to clean fuels and technologies for cooking (% of population)']


interactive(children=(Dropdown(description='Select Detail:', index=4, options=('rural_pop_perc', 'Agriculture,…

In [8]:
# ==============================================================================
# 0. SETUP & INSTALLATION (Ensures Standalone Capability)
# ==============================================================================
import sys
import subprocess
import importlib

# List of required libraries
required_libs = ['pandas', 'numpy', 'plotly', 'scikit-learn', 'ipywidgets', 'statsmodels']

for lib in required_libs:
    if importlib.util.find_spec(lib) is None:
        print(f"Installing missing library: {lib}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", lib])

print("✅ Libraries verified.")

# ==============================================================================
# 1. IMPORTS
# ==============================================================================
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from ipywidgets import interact, widgets
import statsmodels.api as sm

# ==============================================================================
# 2. DATA LOADING & PREPROCESSING
# ==============================================================================
try:
    df = pd.read_csv('../data_cleaned/combined_urbanization_life_quality_2008_2020.csv')
    print(f"✅ Data Loaded: {len(df)} rows.")
except FileNotFoundError:
    print("❌ Error: 'combined_urbanization_life_quality_2008_2020.csv' not found.")
    raise

# Create Country Profiles (Average 2008-2020)
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cols_to_drop = ['Year', 'Country_Code']
profile_features = [c for c in numeric_cols if c not in cols_to_drop]
country_profiles = df.groupby('Country')[profile_features].mean().reset_index()

# ==============================================================================
# 3. COLUMN MAPPING & ORDERING (Strict Sequence Enforced)
# ==============================================================================
# The exact order you requested
indicator_map = {
    'Militarization_Index': 'militarisation',
    'weapons exports': 'weapons exports',
    'weapons imports': 'weapons imports',
    'nuclear and heavy weapons': 'nuclear and heavy weapons',
    'overall score': 'overall score',
    'ongoing conflict': 'ongoing conflict',
    'Neighbouring countries relations': 'Neighbouring countries relations',
    'Political instability': 'Political instability',
    'intensity of internal conflict': 'intensity of internal conflict',
    'Instability_Index': 'internal peace' # Proxy mapping
}

# Capture the exact display order
ordered_display_names = list(indicator_map.keys())

# Create mapping dictionaries
display_to_csv = indicator_map
csv_to_display = {v: k for k, v in indicator_map.items()}

# Identify which CSV columns we actually have
available_csv_cols = []
for k, v in indicator_map.items():
    if v in country_profiles.columns:
        available_csv_cols.append(v)
    else:
        print(f"⚠️ Warning: Column '{v}' not found in dataset. It will be skipped.")

# ==============================================================================
# 4. CLUSTERING (Replicating Logic)
# ==============================================================================
cluster_features = ['urban_pop_perc', 'Gini coefficient (2021 prices)', 'overall score', 'homicide rate']
cluster_data = country_profiles.dropna(subset=cluster_features).copy()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(cluster_data[cluster_features])
kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
cluster_data['Cluster'] = kmeans.fit_predict(X_scaled)

# Assign Labels (Lower Peace Score = Stable)
c0_score = cluster_data[cluster_data['Cluster'] == 0]['overall score'].mean()
c1_score = cluster_data[cluster_data['Cluster'] == 1]['overall score'].mean()

if c0_score < c1_score:
    mapping = {0: 'Stable Urbanizers', 1: 'Volatile Urbanizers'}
else:
    mapping = {1: 'Stable Urbanizers', 0: 'Volatile Urbanizers'}

cluster_data['Cluster_Label'] = cluster_data['Cluster'].map(mapping)
color_map = {'Stable Urbanizers': '#1f77b4', 'Volatile Urbanizers': '#d62728'} # Blue / Red

# Merge
df_final = pd.merge(country_profiles, cluster_data[['Country', 'Cluster_Label']], on='Country', how='inner')

# Create Urban Groups & Enforce Order
urban_order = ['Low (<50%)', 'Medium (50-75%)', 'High (>75%)']
df_final['Urban_Group'] = pd.cut(df_final['urban_pop_perc'], 
                                 bins=[-1, 50, 75, 101], 
                                 labels=urban_order)

# ==============================================================================
# 5. VISUALIZATION 1 PREP (Normalization & Ordering)
# ==============================================================================
vis1_data = df_final[['Urban_Group'] + available_csv_cols].copy()

# MinMax Scaling (0-1) for comparison
min_max_scaler = MinMaxScaler()
vis1_data[available_csv_cols] = min_max_scaler.fit_transform(vis1_data[available_csv_cols])

# Rename columns to Display Names
vis1_data = vis1_data.rename(columns=csv_to_display)

# Melt
melted_vis1 = vis1_data.melt(id_vars='Urban_Group', var_name='Indicator', value_name='Normalized Score')
grouped_vis1 = melted_vis1.groupby(['Urban_Group', 'Indicator'], observed=True)['Normalized Score'].mean().reset_index()

# --- STRICT SORTING ---
# 1. Sort Indicators
grouped_vis1['Indicator'] = pd.Categorical(
    grouped_vis1['Indicator'], 
    categories=ordered_display_names, 
    ordered=True
)
# 2. Sort Urban Groups (Explicitly)
grouped_vis1['Urban_Group'] = pd.Categorical(
    grouped_vis1['Urban_Group'],
    categories=urban_order,
    ordered=True
)
grouped_vis1 = grouped_vis1.sort_values(['Indicator', 'Urban_Group'])

# ==============================================================================
# 6. DASHBOARD GENERATION
# ==============================================================================

# Dropdown options
dropdown_options = [name for name in ordered_display_names if name in grouped_vis1['Indicator'].unique()]

dropdown = widgets.Dropdown(
    options=dropdown_options,
    value='overall score' if 'overall score' in dropdown_options else dropdown_options[0],
    description='Select Detail:',
    style={'description_width': 'initial'}
)

def render_dashboard(selected_display_name):
    # Map back to CSV column name for scatter plot
    csv_col_name = display_to_csv[selected_display_name]

    # --- VISUALIZATION 1: GROUPED BAR CHART (Overview) ---
    fig1 = px.bar(
        grouped_vis1,
        x='Indicator',
        y='Normalized Score',
        color='Urban_Group',
        barmode='group',
        title='<b>1. Landscape Overview: Security Indicators</b><br><i>(Ordered by: Militarization -> Conflict -> Instability)</i>',
        color_discrete_sequence=px.colors.sequential.Viridis,
        # STRICTLY ENFORCE ORDER: Low -> Medium -> High
        category_orders={'Urban_Group': urban_order}, 
        height=500
    )
    # Force X-axis indicator order
    fig1.update_xaxes(categoryorder='array', categoryarray=ordered_display_names)
    fig1.update_layout(xaxis_tickangle=-45, legend_title="Urban Level")
    
    # --- VISUALIZATION 2: SCATTER PLOT (Deep Dive) ---
    fig2 = px.scatter(
        df_final, 
        x='urban_pop_perc', 
        y=csv_col_name, 
        color='Cluster_Label',
        trendline='ols',
        hover_name='Country',
        title=f"<b>2. Deep Dive: {selected_display_name} (Stable vs. Volatile)</b><br><i>(Detailed Correlation Analysis)</i>",
        color_discrete_map=color_map,
        labels={'urban_pop_perc': 'Urban Population (%)', csv_col_name: f'{selected_display_name} (Actual Score)'},
        height=500
    )
    
    # Global Trendline
    global_trend = px.scatter(df_final, x='urban_pop_perc', y=csv_col_name, trendline='ols')
    if len(global_trend.data) > 1:
        trace = global_trend.data[1]
        trace.line.color = 'black'
        trace.line.dash = 'dash'
        trace.name = 'Global Trend'
        trace.showlegend = True
        fig2.add_trace(trace)

    fig2.update_layout(legend_title="Country Cluster")

    # Display
    fig1.show()
    fig2.show()

# Render
print("✨ Generating Dashboard with Ordered Bars (Low->Med->High)...")
interact(render_dashboard, selected_display_name=dropdown);

Installing missing library: scikit-learn...
✅ Libraries verified.
✅ Data Loaded: 767 rows.
✨ Generating Dashboard with Ordered Bars (Low->Med->High)...


interactive(children=(Dropdown(description='Select Detail:', index=4, options=('Militarization_Index', 'weapon…